# Prefix-Tuning 实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

## Step3 数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [5]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Loading cached processed dataset at c:\Users\yuyao\Desktop\transformers\03-PEFT\data\alpaca_data_zh\cache-1be82fe98821990f.arrow


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## Step4 创建模型

In [9]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

In [10]:
sum(param.numel() for param in model.parameters())

1303111680

model size: 1.3B

model: 1.3G * 4 ~= 5.2G

gradient: 1.3G * 4 ~= 5.2G

optimizer: 1.3G * 4 * 2 ~= 10.4G

sum: 20.8G

## Prefix-tuning

### PEFT Step1 配置文件

In [11]:
from peft import PrefixTuningConfig, get_peft_model, TaskType

config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10, prefix_projection=True)
config

PrefixTuningConfig(peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, encoder_hidden_size=None, prefix_projection=True)

### PEFT Step2 创建模型

In [12]:
model = get_peft_model(model, config)

In [13]:
model.prompt_encoder

ModuleDict(
  (default): PrefixEncoder(
    (embedding): Embedding(10, 2048)
    (transform): Sequential(
      (0): Linear(in_features=2048, out_features=2048, bias=True)
      (1): Tanh()
      (2): Linear(in_features=2048, out_features=98304, bias=True)
    )
  )
)

In [14]:
model.print_trainable_parameters()

trainable params: 205,641,728 || all params: 1,508,753,408 || trainable%: 13.629909759249406


## Step5 配置训练参数

In [15]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [17]:
trainer.train()

  0%|          | 0/3357 [00:00<?, ?it/s]

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 3.6547, 'learning_rate': 4.985105749180816e-05, 'epoch': 0.0}
{'loss': 2.7223, 'learning_rate': 4.9702114983616324e-05, 'epoch': 0.01}
{'loss': 2.6773, 'learning_rate': 4.9553172475424484e-05, 'epoch': 0.01}
{'loss': 2.6421, 'learning_rate': 4.940422996723265e-05, 'epoch': 0.01}
{'loss': 2.4778, 'learning_rate': 4.925528745904081e-05, 'epoch': 0.01}
{'loss': 2.3833, 'learning_rate': 4.910634495084897e-05, 'epoch': 0.02}
{'loss': 2.3563, 'learning_rate': 4.895740244265713e-05, 'epoch': 0.02}
{'loss': 2.4198, 'learning_rate': 4.88084599344653e-05, 'epoch': 0.02}
{'loss': 2.3272, 'learning_rate': 4.865951742627346e-05, 'epoch': 0.03}
{'loss': 2.3991, 'learning_rate': 4.851057491808162e-05, 'epoch': 0.03}
{'loss': 2.448, 'learning_rate': 4.836163240988978e-05, 'epoch': 0.03}
{'loss': 2.3553, 'learning_rate': 4.821268990169795e-05, 'epoch': 0.04}
{'loss': 2.4683, 'learning_rate': 4.806374739350611e-05, 'epoch': 0.04}
{'loss': 2.2978, 'learning_rate': 4.791480488531427e-05, 'epoch':

KeyboardInterrupt: 

In [19]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 技巧主要有以下几个方面：\n\n\n1. 掌握考点。在考试前，应认真地预习或复习所考科目相关知识，以便对所考内容不漏点，了解重要考点。根据历年考试的实际情况，考生应重视以下几方面：\n\n- 概念、原理和相关知识的考点\n- 公式、定理、定律、概念的考点\n- 基本技能的考点\n- 考题的类型和难易程度等。\n- 考题中的语言和表达技巧。\n\n2'

## Step8 模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )